In [1]:
import os
import re
import pandas as pd
import copy

# 0. Path
map_data  
&ensp;&ensp;--**provinces.png**

common  
&ensp;&ensp;--bookmarks  
&ensp;&ensp;&ensp;&ensp;--bookmarks  
&ensp;&ensp;--**landed_titles**  
&ensp;&ensp;--**religion**  
&ensp;&ensp;&ensp;&ensp; --**holy_sites**  
&ensp;&ensp;&ensp;&ensp; --**religions**  
&ensp;&ensp;--scripted_effects  

history  
&ensp;&ensp;--characters  
&ensp;&ensp;--titles  


In [ ]:
game_path="F:/SteamLibrary/steamapps/common/Crusader Kings III/game/"
mod_path="C:/Users/XXXXX/Documents/Paradox Interactive/Crusader Kings III/mod/Smaller/"

In [3]:
g_his=game_path+"history/"
g_cha=g_his+"characters/"
g_ti=g_his+"titles/"
g_landtitles= game_path+"common/landed_titles/00_landed_titles.txt"
g_holy_cites=game_path+"common/religion/holy_sites/00_holy_sites.txt"
g_religions=game_path+"common/religion/religions/"
g_religions_files=sorted([g_religions+i for i in os.listdir(g_religions) if i[-3:]=="txt"])
g_bookmark=game_path+"common/bookmarks/bookmarks/00_bookmarks.txt"

m_his=mod_path+"history/"
m_cha=m_his+"characters/"
m_ti=m_his+"titles/"
m_landtitles= mod_path+"common/landed_titles/00_landed_titles.txt"
m_holy_cites=mod_path+"common/religion/holy_sites/00_holy_sites.txt"
m_religions=mod_path+"common/religion/religions/"
m_religions_files=sorted([m_religions+i for i in os.listdir(g_religions) if i[-3:]=="txt"])
m_bookmark=mod_path+"common/bookmarks/bookmarks/00_bookmarks.txt"

# 1. Parse

In [4]:
def tokenize(file):
    with open(file, "r", encoding='utf-8-sig') as a:
        lines=[]
        lines_=a.readlines()
        for i in lines_:
            if i and i[0] not in ["@","#"]:
                s=re.sub(r"#.*", "", i).strip()
                if s:
                    s=s.replace("hsv{","{")
                    s_lst=re.split(r'(=|\{|\})', s)
                    for j in s_lst:
                        if j.strip():
                            lines.append(j.strip())
    
    result = []
    i = 0
    while i < len(lines):
        if lines[i] == '=' and i + 1 < len(lines) and lines[i+1] == '{':
            result.append('={')
            i += 2  # 跳过 '=' 和 '{'
        else:
            result.append(lines[i])
            i += 1
    
    return result

def token_to_lst(token_lst):
    t_lst=[]
    eqal_flag=False
    for t in token_lst:
        if t=="=":
            eqal_flag=True
        elif t=="={":
            t_lst.append(t)
        elif t=="}":
            temp_lst=[]
            while (t_:=t_lst.pop())!="={":
                temp_lst.append(t_)
    
            t_lst.append({t_lst.pop():temp_lst})
            temp_lst=[]
        else:
            if eqal_flag==False:
                t_lst.append(t)
            else:
                t_lst.append({t_lst.pop():t})
                eqal_flag=False
    return t_lst

def merge_token_list(data):
    if isinstance(data, list):
        if len(data)==1 and isinstance(data[0], str):
            return data[0]
            
        elif all(isinstance(item, dict) for item in data):
            merged = {}
            for d in data:
                for k, v in d.items():
                    merged[k] = merge_token_list(v)
            return merged
        else:
            # 是一个普通列表（比如 ['a', 'b', 'c']），直接返回，或递归处理
            return [merge_token_list(item) for item in data]
    elif isinstance(data, dict):
        # 如果是字典，则递归处理每一个 value
        return {k: merge_token_list(v) for k, v in data.items()}
    else:
        # 原子值，直接返回
        return data

def file_to_dict(path):
    tokens=tokenize(path)
    token_list=token_to_lst(tokens)
    dic=merge_token_list(token_list)
    return dic

def keep_prefix(dic, prefix):
    lst=[]
    for i,j in dic.items():
        if i[:2] not in prefix:
            lst.append(i)
        else:
            if type(j)==dict:
                j = keep_prefix(j,prefix)
    for i in lst:
        del dic[i]
    return dic

In [39]:
title_delete=[
"e_ajuraan",
"e_bengal",
"e_deccan",
"e_guinea",
"e_mali",
"e_kanem_bornu",
"e_tibet",
"e_rajastan",
"e_abyssinia",

"k_anbiya",
"k_himalaya",
"k_khotan",
"k_permia",
"k_sahara",
"k_yugra",
]

In [40]:
dic=file_to_dict(g_landtitles)
dic=keep_prefix(dic, ["e_","k_","d_","c_"])

In [141]:
rows = []
for i,j in dic.items():
    if i[:2]=="e_":
        for kingdom, duchies in j.items():
            for duchy, counties in duchies.items():
                for county in counties:
                    rows.append((i,kingdom, duchy, county))
df = pd.DataFrame(rows, columns=["empire", "kingdom", "duchy", "county"])
not_exist=df[df.isin(title_delete).any(axis=1)]
not_exist_all=not_exist.stack().unique().tolist()
not_exist_county_list=not_exist.county.unique().tolist()

# 2. land_titles

In [119]:
replace_title={
    "capital = c_sagya":"capital = c_riqan",
    "capital = c_shazhou":"capital = c_bost",
}

In [128]:
def delete_key(from_path, lst, rpls,to_path=None):
    with open(from_path, "r", encoding="utf-8-sig") as f:
        s=f.read()
    
    for key in lst:
        if len(s)==0:
            break
        index=s.find(key)
        if index!=-1:
            l,r,offset=0,0,0
            while l==0 or l!=r:
                offset+=1
                if s[index+offset]=="{":
                    l+=1
                elif s[index+offset]=="}":
                    r+=1
            s=s[:index]+s[index+offset+1:]

    if rpls:
        for i,j in rpls.items():
            s=s.replace(i,j)
    
    if not to_path:
        return s
    
    with open(to_path, "w", encoding="utf-8-sig") as f:
        f.write(s)


In [44]:
delete_key(g_landtitles,title_delete,replace_title, m_landtitles)

# 3. holy_sites

In [45]:
dic_hc=file_to_dict(g_holy_cites)
not_exist_holy_cites=[i for i,j in dic_hc.items() if j["county"] in not_exist_county_list]

In [46]:
delete_key(g_holy_cites,[i+" =" for i in not_exist_holy_cites],None, m_holy_cites) #.replace(".txt","_test.txt")

In [47]:
new_hc=file_to_dict(m_holy_cites)

In [150]:
rlps={
    "bono":"aswan",
    "kusinagara":"nok_kundi",
    "itanagar":"nok_kundi",
    "varanasi":"nok_kundi",
    "palitana":"nok_kundi",
    "kathmandu":"nok_kundi",
    "ngawa":"kara_khorum",
    "lhasa":"kara_khorum",
    "danakil":"carthage",
    "awkar":"aswan",
    "daura":"aswan",
    "ife":"tinmallal",
    "kisi":"tinmallal",
}

def comment_sites(from_lst,to_lst,not_exist_holy_cites):
    sites=[f"holy_site = {i}" for i in not_exist_holy_cites]
    for i,j in zip(from_lst,to_lst):
        with open(i, "r", encoding="utf-8-sig") as f:
            s=f.read()
            copy_s=copy.copy(s)
            for m,n in rlps.items():
                s=s.replace(f"holy_site = {m}",f"holy_site = {n}")
            
            for k in sites:
                s=s.replace(k,"#"+k)
        if s!=copy_s:
            with open(j, "w", encoding="utf-8-sig") as f:
                f.write(s)

In [151]:
comment_sites(g_religions_files,m_religions_files,not_exist_holy_cites)

# 4. bookmarks

In [130]:
dic_bkmk=file_to_dict(g_bookmark)

In [ ]:
def remove_character(dic):
    lst=[]
    for i,j in dic.items():
        if i=="character":
            if i["title"] in not_exist_all:
                lst.append(i)
            else:
                
        else:
            if type(j)==dict:
                lst+=remove_character(dic)
    

    return lst

In [144]:
dic_bkmk["bm_867_adventurers"]["character"]

{'character': {'history_id': '6880',
  'religion': 'norse_pagan',
  'culture': 'norse',
  'birth': '860.1.1',
  'type': 'male',
  'dynasty': '1059',
  'relation': '"BOOKMARK_RELATION_SON"',
  'name': '"bookmark_adventurers_jarl_haesteinn_alt_ragnarr"'},
 'animation': 'personality_rational',
 'position': '320 400',
 'history_id': '6878',
 'difficulty': '"BOOKMARK_CHARACTER_DIFFICULTY_EASY"',
 'religion': 'norse_pagan',
 'culture': 'norse',
 'government': 'feudal_government',
 'title': 'c_montaigu',
 'birth': '815.1.1',
 'type': 'male',
 'dynasty_splendor_level': '1',
 'dynasty': '1059',
 'name': '"bookmark_adventurers_jarl_haesteinn"'}